In [ ]:
%pip -q install -r colab/Requirements.txt

In [ ]:
#Imports
import os
import getpass
import re
import time
import unicodedata
from smolagents import ToolCallingAgent, OpenAIServerModel, tool

# Configuración

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass.getpass("Introduce OPENAI_API_KEY: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
MODEL_ID = os.getenv("OPENAI_MODEL", "gpt-4o-mini")


def build_model():
    return OpenAIServerModel(model_id=MODEL_ID, api_key=OPENAI_API_KEY)

#Memoria y utilidades de texto

class EvidenceStore:
    _data = {}

    @classmethod
    def reset(cls):
        cls._data = {}

    @classmethod
    def add(cls, key: str, value: str):
        cls._data[key] = value

    @classmethod
    def get(cls):
        return dict(cls._data)

def _norm(s: str) -> str:
    s = s.lower().strip()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")
    return s

    # Validar formato de salida en formato aviso
def check_veredicto_completo(final_answer: str):
    if not isinstance(final_answer, str):
        raise ValueError("La salida final no es texto.")
    if not re.search(r"(?im)^\s*veredicto\s*:\s*(.+)$", final_answer):
        raise ValueError("Falta 'Veredicto: ...'.")
    if not re.search(r"(?im)^\s*justificaci[oó]n\s+breve\s*:\s*(.+)$", final_answer):
        raise ValueError("Falta 'Justificación breve: ...'.")
    if not re.search(r"(?im)^\s*confiabilidad\s*:\s*(0(\.\d+)?|1(\.0+)?)\s*$", final_answer):
        raise ValueError("Falta 'Confiabilidad: 0.0–1.0'.")
    return True

def _extract_veredicto(s: str):
    pat_es = r"(?im)^[\s\*\-_>]*veredicto\s*:\s*(verdadero|falso|dudoso)\b"
    m = list(re.finditer(pat_es, s))
    if m:
        return m[-1].group(1).lower()

    pat_en = r"(?im)^[\s\*\-_>]*(verdict|decision)\s*:\s*(true|false|uncertain|inconclusive)\b"
    m = list(re.finditer(pat_en, s))
    if m:
        word = m[-1].group(2).lower()
        return {"true":"verdadero","false":"falso","uncertain":"dudoso","inconclusive":"dudoso"}[word]

    return None

def _extract_confiabilidad(s: str):
    num = r"(0(?:[.,]\d+)?|1(?:[.,]0+)?)"
    m = re.findall(rf"(?im)^\s*confiabilidad\s*:\s*{num}\s*$", s)
    if m:
        return m[-1].replace(",", ".")

    m = re.findall(rf"(?im)\bconfiabilidad\s*:\s*{num}\b", s)
    if m:
        return m[-1].replace(",", ".")

    return None


def _extract_justificacion(s: str):
    num = r"(0(?:[.,]\d+)?|1(?:[.,]0+)?)"
    pat = rf"(?im)^[\s\*\-_>]*justificaci[oó]n\s+breve\s*:\s*(.+?)(?:\s+confiabilidad\s*:\s*{num}\s*)?$"
    m = re.search(pat, s)
    return m.group(1).strip() if m else None

#Prompts fijos

SYSTEM_PROMPT_JURADO = (
    "Eres un jurado de IA. Tu única tarea es decidir si una afirmación es "
    "VERDADERA, FALSA o DUDOSA basándote en los análisis aportados.\n\n"
    "Devuelve SIEMPRE en el siguiente formato EXACTO (en español):\n\n"
    "Veredicto: <Verdadero|Falso|Dudoso>\n"
    "Justificación breve: <máx. 3 frases, concretas>\n"
    "Confiabilidad: <número entre 0.0 y 1.0>\n"
)

SUB_PROMPT_SENSACIONALISMO = (
    "Analiza si la afirmación usa lenguaje sensacionalista/emocional. "
    "Devuelve 2-4 frases, objetivas y concisas."
)
SUB_PROMPT_GRAMATICA = (
    "Revisa la afirmación y detecta errores gramaticales/ortográficos/estilo. "
    "Devuelve 2-4 frases, claras y útiles que indiquen si hay errores gramaticales/ortográficos/estilo o por lo contrario si esta correctamente escrita la afirmación."
)
SUB_PROMPT_SENTIDO_COMUN = (
    "Evalúa si la afirmación contradice el sentido común. "
    "Devuelve 2-4 frases, con razonamiento breve."
)

# Crear Agentes
def make_agent(instructions: str, tools=None, max_steps: int = 6) -> ToolCallingAgent:
    tools = tools or []
    try:
        return ToolCallingAgent(
            model=build_model(),
            tools=tools,
            instructions=instructions,
            max_steps=max_steps,
        )
    except TypeError:
        try:
            return ToolCallingAgent(
                model=build_model(),
                tools=tools,
                instructions=instructions,
            )
        except TypeError:
            agent = ToolCallingAgent(model=build_model(), tools=tools)
            if hasattr(agent, "instructions"):
                agent.instructions = instructions
            elif hasattr(agent, "system_prompt"):
                agent.system_prompt = instructions
            return agent

def run_with_retry(agent: ToolCallingAgent, prompt: str, retries: int = 2, backoff: float = 1.0) -> str:
    for i in range(retries + 1):
        try:
            return agent.run(prompt)
        except Exception:
            if i == retries:
                raise
            time.sleep(backoff * (i + 1))

#Subagentes/Tools

@tool
def evaluar_sentido_comun(texto: str) -> str:
    """Evalúa si la afirmación contradice el sentido común.
    Args:
        texto (str): La afirmación a evaluar.
    Returns:
        str: Análisis breve (2–4 frases).
    """
    out = subagente_texto(SUB_PROMPT_SENTIDO_COMUN, texto)
    EvidenceStore.add("sentido_comun", out)
    return out

@tool
def evaluar_sensacionalismo(texto: str) -> str:
    """Detecta uso de lenguaje sensacionalista o emocional.
    Args:
        texto (str): La afirmación a evaluar.
    Returns:
        str: Análisis breve (2–4 frases).
    """
    out = subagente_texto(SUB_PROMPT_SENSACIONALISMO, texto)
    EvidenceStore.add("sensacionalismo", out)
    return out

@tool
def evaluar_gramatica(texto: str) -> str:
    """Revisa gramática, ortografía y estilo.
    Args:
        texto (str): La afirmación a evaluar.
    Returns:
        str: Análisis breve (2–4 frases).
    """
    out = subagente_texto(SUB_PROMPT_GRAMATICA, texto)
    EvidenceStore.add("gramatica", out)
    return out

def subagente_texto(instrucciones: str, afirmacion: str) -> str:
    agent = make_agent(instrucciones, tools=[])
    return run_with_retry(agent, f"Afirmación: {afirmacion}", retries=1, backoff=0.8)

# Analizar afirmación
def analizar_afirmacion(claim: str) -> str:
    claim = str(claim).strip()
    if not claim:
        raise ValueError("La afirmación está vacía.")

    EvidenceStore.reset()

    jurado = make_agent(
        SYSTEM_PROMPT_JURADO + (
            "\n\nDEBES recopilar evidencia llamando a herramientas antes del veredicto. "
            "Llama a (1) evaluar_sensacionalismo, (2) evaluar_gramatica y "
            "(3) evaluar_sentido_comun sobre la MISMA afirmación, y solo después dicta veredicto."
        ),
        tools=[evaluar_sensacionalismo, evaluar_gramatica, evaluar_sentido_comun],
        max_steps=12,
    )

    try:
        jurado.verbose = False
    except Exception:
        pass

    prompt_veredicto = (
        f'Afirmación: "{claim}"\n\n'
        "Primero, usa las herramientas indicadas con el texto de la afirmación. "
        "Cuando tengas suficiente evidencia, responde únicamente en este formato EXACTO:\n\n"
        "Veredicto: <Verdadero|Falso|Dudoso>\n"
        "Justificación breve: <máx. 3 frases, concretas>\n"
        "Confiabilidad: <número entre 0.0 y 1.0>\n"
        "No incluyas 'Confiabilidad' dentro de 'Justificación breve'."
    )


    salida = run_with_retry(jurado, prompt_veredicto, retries=2, backoff=0.8)

    aviso_validator = ""
    try:
        check_veredicto_completo(salida)
    except Exception as e:
        aviso_validator = f"\n\n> **Aviso del validador**: {e}"

    veredicto_final     = _extract_veredicto(salida) or "—"
    justificacion_final = _extract_justificacion(salida) or "—"
    confianza_final     = _extract_confiabilidad(salida) or "—"


    evid = EvidenceStore.get()
    evid_sens = evid.get("sensacionalismo", "— (no se invocó la tool)")
    evid_gram = evid.get("gramatica", "— (no se invocó la tool)")
    evid_comn = evid.get("sentido_comun", "— (no se invocó la tool)")

    # Markdown final
    resultado_md = (
        f"**Veredicto:** {veredicto_final}\n\n"
        f"**Justificación breve:** {justificacion_final}\n\n"
        f"**Confiabilidad:** {confianza_final}{aviso_validator}\n\n"
        "---\n\n"
        f"### Evidencias de subagentes\n"
        f"- **Sensacionalismo:** {evid_sens}\n"
        f"- **Gramática:** {evid_gram}\n"
        f"- **Sentido común:** {evid_comn}\n"
    )

    return resultado_md

In [ ]:
resultado = analizar_afirmacion("El agujero en la capa de ozono sobre la Antártida se detectó por primera vez en los años 80.")
print(resultado)


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
#Imports
import re, time
import pandas as pd
import tiktoken
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

#Evaluación y rendimiento
try:
    ENC = tiktoken.get_encoding("o200k_base")
except Exception:
    ENC = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    return len(ENC.encode(text or ""))

MAPEO = {
    "verdadero":"Verdadero","true":"Verdadero","real":"Verdadero","cierto":"Verdadero",
    "correcto":"Verdadero","veraz":"Verdadero","no es falso":"Verdadero","no es fake":"Verdadero",
    "no fake":"Verdadero","not false":"Verdadero","not fake":"Verdadero",
    "falso":"Falso","false":"Falso","fake":"Falso","fake news":"Falso","engaño":"Falso",
    "hoax":"Falso","incorrecto":"Falso",
    "dudoso":"Dudoso","dudosa":"Dudoso","incierto":"Dudoso","inconcluso":"Dudoso",
    "uncertain":"Dudoso","inconclusive":"Dudoso","half-true":"Dudoso","rumor":"Dudoso"
}
def mapeo(x: str) -> str:
    s = str(x).strip().lower()
    return MAPEO.get(s, s.title())


def extraer_md_veredicto_vfd(md_text: str) -> str:
    if not isinstance(md_text, str):
        return "Dudoso"
    pat = r"(?im)^[\s>*-]*\**\s*veredicto\s*:\s*\**\s*([a-záéíóúüñ]+)"
    m = re.search(pat, md_text or "")
    if not m:
        return "Dudoso"
    v = m.group(1).strip().lower().rstrip(".")
    if v.startswith("verdader"): return "Verdadero"
    if v.startswith("falso") or v.startswith("fake"): return "Falso"
    if v.startswith("dudoso") or v.startswith("dudosa"): return "Dudoso"
    return "Dudoso"

def extraer_md_confianza(md_text: str) -> str:
    num = r"(0(?:[.,]\d+)?|1(?:[.,]0+)?)"
    m = re.search(rf"(?im)^[\s>*-]*\**\s*confiabilidad\s*:\s*\**\s*{num}\b", md_text or "")
    return m.group(1).replace(",", ".") if m else "—"

EXCEL_PATH = "/colab/data/dataset_pruebas.xlsx"
OUTPUT_CSV = "/colab/data/resultados_con_evidencias.csv"

df_eval = pd.read_excel(EXCEL_PATH)
assert "Afirmación" in df_eval.columns, "No encuentro la columna 'Afirmación'."
assert "Categoria" in df_eval.columns, "No encuentro la columna 'Categoria'."
df_eval["y_true"] = df_eval["Categoria"].apply(mapeo)

preds, confs = [], []
times, in_toks, out_toks, tot_toks = [], [], [], []

for i, texto in enumerate(df_eval["Afirmación"], start=1):
    texto = str(texto)

    t0 = time.perf_counter()
    md = analizar_afirmacion(texto)
    elapsed = time.perf_counter() - t0
    times.append(round(elapsed, 3))


    preds.append(extraer_md_veredicto_vfd(md))
    confs.append(extraer_md_confianza(md))

    in_est = count_tokens(f'Afirmación: "{texto}"')
    try:
        in_est += count_tokens(SYSTEM_PROMPT_JURADO)
        in_est += count_tokens(SUB_PROMPT_SENSACIONALISMO + f"Afirmación: {texto}")
        in_est += count_tokens(SUB_PROMPT_GRAMATICA + f"Afirmación: {texto}")
        in_est += count_tokens(SUB_PROMPT_SENTIDO_COMUN + f"Afirmación: {texto}")
    except NameError:
        pass
    out_est = count_tokens(md)

    in_toks.append(in_est)
    out_toks.append(out_est)
    tot_toks.append(in_est + out_est)

    if i % 10 == 0:
        print(f"Procesadas {i} filas...")

# Guardar resultados mínimos
df_eval["y_pred"] = preds
df_eval["confiabilidad"] = confs
df_eval["tiempo_s"] = times
df_eval["input_tokens_est"] = in_toks
df_eval["output_tokens_est"] = out_toks
df_eval["total_tokens_est"] = tot_toks

df_eval.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"Resultados guardados en: {OUTPUT_CSV}")

#Métricas básicas
labels_vfd = ["Falso", "Dudoso", "Verdadero"]
print("\n=== Accuracy ===", accuracy_score(df_eval["y_true"], df_eval["y_pred"]))
print("\n=== Classification report ===")
print(classification_report(df_eval["y_true"], df_eval["y_pred"],
                            labels=labels_vfd, target_names=labels_vfd, zero_division=0))

# 5)Tiempo y tokens por afirmación
ids = df_eval.index + 1

plt.figure(figsize=(14,6))
plt.bar(ids, df_eval["tiempo_s"])
plt.title("Duración total por afirmación (s)")
plt.xlabel("ID de afirmación")
plt.ylabel("Tiempo (segundos)")
plt.xticks(ids)
plt.show()

plt.figure(figsize=(14,6))
plt.bar(ids, df_eval["total_tokens_est"])
plt.title("Tokens procesados por afirmación (total estimado)")
plt.xlabel("ID de afirmación")
plt.ylabel("Tokens")
plt.xticks(ids)
plt.show()

